In [ ]:
!pip install tensorflow
!pip install mediapipe
import pandas as pd
import cv2
import numpy as np
import os
import pandas as pd
import cv2
import mediapipe as mp
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow.keras.models import load_model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
i3d_model_url = "https://tfhub.dev/deepmind/i3d-kinetics-400/1"
i3d_model = hub.load(i3d_model_url)
example_input = tf.random.normal([1, 64, 224, 224, 3])
output = i3d_model(example_input)
print(output.shape)

TypeError: 'AutoTrackable' object is not callable

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def inception_module(x, filters):

    branch_1 = layers.Conv3D(filters[0], kernel_size=(1, 1, 1), padding='same', activation='relu')(x)
    branch_2 = layers.Conv3D(filters[1], kernel_size=(1, 1, 1), padding='same', activation='relu')(x)
    branch_2 = layers.Conv3D(filters[2], kernel_size=(3, 3, 3), padding='same', activation='relu')(branch_2)
    branch_3 = layers.Conv3D(filters[3], kernel_size=(1, 1, 1), padding='same', activation='relu')(x)
    branch_3 = layers.Conv3D(filters[4], kernel_size=(5, 5, 5), padding='same', activation='relu')(branch_3)
    branch_4 = layers.MaxPooling3D(pool_size=(3, 3, 3), strides=(1, 1, 1), padding='same')(x)
    branch_4 = layers.Conv3D(filters[5], kernel_size=(1, 1, 1), padding='same', activation='relu')(branch_4)

    # Concatenate all branches
    x = layers.concatenate([branch_1, branch_2, branch_3, branch_4], axis=-1)
    return x

def I3D_model(input_shape=(16, 64, 64, 3), num_classes=2):
    inputs = layers.Input(shape=input_shape)

    x = layers.Conv3D(64, kernel_size=(7, 7, 7), strides=(2, 2, 2), padding='same', activation='relu')(inputs)
    x = layers.MaxPooling3D(pool_size=(1, 3, 3), strides=(1, 2, 2), padding='same')(x)

    x = layers.Conv3D(64, kernel_size=(1, 1, 1), padding='same', activation='relu')(x)
    x = layers.Conv3D(192, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = layers.MaxPooling3D(pool_size=(1, 3, 3), strides=(1, 2, 2), padding='same')(x)

    x = inception_module(x, [64, 96, 128, 16, 32, 32])
    x = inception_module(x, [128, 128, 192, 32, 96, 64])
    x = layers.MaxPooling3D(pool_size=(3, 3, 3), strides=(2, 2, 2), padding='same')(x)

    x = inception_module(x, [192, 96, 208, 16, 48, 64])
    x = inception_module(x, [160, 112, 224, 24, 64, 64])
    x = inception_module(x, [128, 128, 256, 24, 64, 64])
    x = inception_module(x, [112, 144, 288, 32, 64, 64])
    x = inception_module(x, [256, 160, 320, 32, 128, 128])
    x = layers.MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)

    x = inception_module(x, [256, 160, 320, 32, 128, 128])
    x = inception_module(x, [384, 192, 384, 48, 128, 128])

    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs, outputs)
    return model


input_shape = (16, 64, 64, 3)  # Example shape for input (16 frames, 64x64 resolution, 3 channels)
num_classes = 2
model = I3D_model(input_shape=input_shape, num_classes=num_classes)


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 16, 64, 64, 3)  │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d (Conv3D)           │ (None, 8, 32, 32, 64)  │         65,920 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling3d             │ (None, 8, 16, 16, 64)  │              0 │ conv3d[0][0]           │
│ (MaxPooling3D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_1 (Conv3D)         │ (None, 8, 16, 16, 64)  │          4,160 │ max_pooling3d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_2 (Conv3D)         │ (None, 8, 16, 16, 192) │        331,968 │ conv3d_1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling3d_1           │ (None, 8, 8, 8, 192)   │              0 │ conv3d_2[0][0]         │
│ (MaxPooling3D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_4 (Conv3D)         │ (None, 8, 8, 8, 96)    │         18,528 │ max_pooling3d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_6 (Conv3D)         │ (None, 8, 8, 8, 16)    │          3,088 │ max_pooling3d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling3d_2           │ (None, 8, 8, 8, 192)   │              0 │ max_pooling3d_1[0][0]  │
│ (MaxPooling3D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_3 (Conv3D)         │ (None, 8, 8, 8, 64)    │         12,352 │ max_pooling3d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_5 (Conv3D)         │ (None, 8, 8, 8, 128)   │        331,904 │ conv3d_4[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_7 (Conv3D)         │ (None, 8, 8, 8, 32)    │         64,032 │ conv3d_6[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_8 (Conv3D)         │ (None, 8, 8, 8, 32)    │          6,176 │ max_pooling3d_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 8, 8, 8, 256)   │              0 │ conv3d_3[0][0],        │
│                           │                        │                │ conv3d_5[0][0],        │
│                           │                        │                │ conv3d_7[0][0],        │
│                           │                        │                │ conv3d_8[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_10 (Conv3D)        │ (None, 8, 8, 8, 128)   │         32,896 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_12 (Conv3D)        │ (None, 8, 8, 8, 32)    │          8,224 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling3d_3      

 Total params: 14,615,218 (55.75 MB)

 Trainable params: 14,615,218 (55.75 MB)

 Non-trainable params: 0 (0.00 B)

Num GPUs Available: 1


In [ ]:

def create_annotations_csv(dataset_dir, output_csv):
    data = []

    # Define label mapping
    labels = {"Healthy": 0, "Injured": 1}

    # Traverse through the 'healthy' and 'injured' directories
    for label_name, label_value in labels.items():
        label_dir = os.path.join(dataset_dir, label_name)
        for video_file in os.listdir(label_dir):
            if video_file.endswith('.mp4'):
                video_path = os.path.join(label_dir, video_file)
                data.append([video_path, label_value])

    # Create a DataFrame and save it as a CSV
    df = pd.DataFrame(data, columns=["filepath", "label"])
    df.to_csv(output_csv, index=False)
    print(f"Annotation CSV created at {output_csv}")

# Path to your dataset directory and desired CSV output path
dataset_dir = '/content/drive/My Drive/Final_Dataset'
output_csv = os.path.join(dataset_dir, 'annotations.csv')

# Create the annotation CSV
create_annotations_csv(dataset_dir, output_csv)

In [ ]:



data = pd.read_csv('/content/drive/My Drive/Final_Dataset/annotations.csv')
print(data['label'].value_counts())


train_dev_data, test_data = train_test_split(
    data, test_size=0.2, stratify=data['label'], random_state=42
)


train_data, dev_data = train_test_split(
    train_dev_data, test_size=0.25, stratify=train_dev_data['label'], random_state=42
)

# Check class distribution in each set
print("Train set class distribution:\n", train_data['label'].value_counts())
print("Dev set class distribution:\n", dev_data['label'].value_counts())
print("Test set class distribution:\n", test_data['label'].value_counts())

train_data.to_csv('/content/drive/My Drive/Final_Dataset/train.csv', index=False)
dev_data.to_csv('/content/drive/My Drive/Final_Dataset/dev.csv', index=False)
test_data.to_csv('/content/drive/My Drive/Final_Dataset/test.csv', index=False)

label
0    9565
1    3609
Name: count, dtype: int64
Train set class distribution:
 label
0    5739
1    2165
Name: count, dtype: int64
Dev set class distribution:
 label
0    1913
1     722
Name: count, dtype: int64
Test set class distribution:
 label
0    1913
1     722
Name: count, dtype: int64


In [ ]:


def preprocess_video(video_path, num_frames=16, frame_size=(64, 64)):
    """
    Preprocess a video by resizing frames and reducing the number of frames.

    Arguments:
    video_path -- Path to the video file.
    num_frames -- Number of frames to sample from the video.
    frame_size -- Resolution to resize frames (width, height).

    Returns:
    video_tensor -- A numpy array of shape (num_frames, frame_size[0], frame_size[1], 3).
    """
    cap = cv2.VideoCapture(video_path)
    frames = []
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate frame sampling step to evenly sample num_frames from the video
    step = max(1, frame_count // num_frames)

    for i in range(num_frames):
        frame_index = i * step
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)  # Move to the selected frame
        ret, frame = cap.read()

        if not ret:
            break

        # Resize the frame to the specified size
        frame = cv2.resize(frame, frame_size)
        frames.append(frame)

    cap.release()

    # If the video has fewer frames than num_frames, pad with black frames
    while len(frames) < num_frames:
        frames.append(np.zeros((frame_size[1], frame_size[0], 3), dtype=np.uint8))

    # Normalize pixel values to [0, 1]
    video_tensor = np.array(frames, dtype=np.float32) / 255.0
    return video_tensor


In [ ]:


def extract_pose_data_from_video(video_path, num_keypoints=33):
    """
    Extracts pose keypoints from a video using MediaPipe Pose.

    Arguments:
    video_path -- Path to the input video.
    num_keypoints -- Number of keypoints expected from MediaPipe Pose.

    Returns:
    pose_data -- A list of pose keypoints for each frame.
    """
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    cap = cv2.VideoCapture(video_path)
    pose_data = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame to get pose landmarks
        results = pose.process(image)

        if results.pose_landmarks:
            # Extract the pose landmarks (x, y, z coordinates)
            frame_data = []
            for landmark in results.pose_landmarks.landmark:
                frame_data.extend([landmark.x, landmark.y, landmark.z])

            # Append the frame's data to the dataset
            pose_data.append(frame_data)
        else:
            # If no pose detected, append a vector of zeros
            pose_data.append([0] * num_keypoints * 3)

    cap.release()
    return pose_data

def process_excel_files(input_folder, output_folder, num_keypoints=33):
    """
    Processes all Excel files in the input folder, extracts pose keypoints, and saves results to output folder.

    Arguments:
    input_folder -- Path to the folder containing Excel files with video paths.
    output_folder -- Path to save processed Excel files with pose keypoints.
    num_keypoints -- Number of keypoints expected from MediaPipe Pose.
    """
    os.makedirs(output_folder, exist_ok=True)

    for excel_file in os.listdir(input_folder):
        if excel_file.endswith('.xlsx') or excel_file.endswith('.xls'):
            input_path = os.path.join(input_folder, excel_file)
            print(f"Processing {excel_file}...")

            # Load the Excel file
            df = pd.read_excel(input_path)

            # Check if there is a 'path' column with video paths
            if 'path' not in df.columns:
                print(f"Skipping {excel_file}: No 'path' column found.")
                continue

            all_pose_data = []
            for _, row in df.iterrows():
                video_path = row['path']
                label = row.get('label', None)  # Optional label column

                if os.path.exists(video_path):
                    pose_data = extract_pose_data_from_video(video_path, num_keypoints)
                    for frame_data in pose_data:
                        if label is not None:
                            all_pose_data.append([video_path, label] + frame_data)
                        else:
                            all_pose_data.append([video_path] + frame_data)
                else:
                    print(f"Video not found: {video_path}")

            # Save the pose data to a new Excel file
            columns = ['path'] + (['label'] if 'label' in df.columns else []) + \
                      [f'keypoint_{i}_{axis}' for i in range(num_keypoints) for axis in ['x', 'y', 'z']]
            output_df = pd.DataFrame(all_pose_data, columns=columns)

            output_file = os.path.join(output_folder, f"processed_{excel_file}")
            output_df.to_excel(output_file, index=False)
            print(f"Saved processed data to {output_file}")


process_excel_files(input_folder, output_folder)


FileNotFoundError: [Errno 2] No such file or directory: '/path/to/your/input_excels'

In [ ]:
def data_generator(file_paths, labels, preprocess_func, batch_size=16):
    """
    Generator to yield batches of preprocessed data and labels.

    Arguments:
    file_paths -- List of file paths.
    labels -- Corresponding list of labels.
    preprocess_func -- Function to preprocess a single file.
    batch_size -- Number of samples per batch.

    Yields:
    batch_x -- Batch of preprocessed data.
    batch_y -- Batch of corresponding labels.
    """
    num_samples = len(file_paths)
    while True:
        for i in range(0, num_samples, batch_size):
            batch_paths = file_paths[i:i + batch_size]
            batch_labels = labels[i:i + batch_size]

            # Preprocess each file
            batch_x = [preprocess_func(file_path) for file_path in batch_paths]
            batch_x = tf.stack(batch_x)  # Stack into a tensor

            batch_y = tf.convert_to_tensor(batch_labels)

            yield batch_x, batch_y



In [ ]:
model_path = '/content/drive/My Drive/Final_Dataset/saved_model/my_model.h5'
model= load_model(model_path)

# Confirm the model is loaded
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 16, 64, 64, 3)  │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d (Conv3D)           │ (None, 8, 32, 32, 64)  │         65,920 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling3d             │ (None, 8, 16, 16, 64)  │              0 │ conv3d[0][0]           │
│ (MaxPooling3D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_1 (Conv3D)         │ (None, 8, 16, 16, 64)  │          4,160 │ max_pooling3d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_2 (Conv3D)         │ (None, 8, 16, 16, 192) │        331,968 │ conv3d_1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling3d_1           │ (None, 8, 8, 8, 192)   │              0 │ conv3d_2[0][0]         │
│ (MaxPooling3D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_4 (Conv3D)         │ (None, 8, 8, 8, 96)    │         18,528 │ max_pooling3d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_6 (Conv3D)         │ (None, 8, 8, 8, 16)    │          3,088 │ max_pooling3d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling3d_2           │ (None, 8, 8, 8, 192)   │              0 │ max_pooling3d_1[0][0]  │
│ (MaxPooling3D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_3 (Conv3D)         │ (None, 8, 8, 8, 64)    │         12,352 │ max_pooling3d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_5 (Conv3D)         │ (None, 8, 8, 8, 128)   │        331,904 │ conv3d_4[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_7 (Conv3D)         │ (None, 8, 8, 8, 32)    │         64,032 │ conv3d_6[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_8 (Conv3D)         │ (None, 8, 8, 8, 32)    │          6,176 │ max_pooling3d_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 8, 8, 8, 256)   │              0 │ conv3d_3[0][0],        │
│                           │                        │                │ conv3d_5[0][0],        │
│                           │                        │                │ conv3d_7[0][0],        │
│                           │                        │                │ conv3d_8[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_10 (Conv3D)        │ (None, 8, 8, 8, 128)   │         32,896 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_12 (Conv3D)        │ (None, 8, 8, 8, 32)    │          8,224 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling3d_3      

 Total params: 14,615,220 (55.75 MB)

 Trainable params: 14,615,218 (55.75 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [ ]:

data_directory = '/content/drive/My Drive/Final_Dataset'

# Load training and validation file paths and labels
train_df = pd.read_csv(os.path.join(data_directory, 'train.csv'))
dev_df = pd.read_csv(os.path.join(data_directory, 'dev.csv'))

train_paths = train_df['filepath'].tolist()
train_labels = train_df['label'].tolist()
dev_paths = dev_df['filepath'].tolist()
dev_labels = dev_df['label'].tolist()

test_df = pd.read_csv(os.path.join(data_directory, 'test.csv'))
test_paths = test_df['filepath'].tolist()
test_labels = test_df['label'].tolist()
test_gen = data_generator(test_paths, test_labels, preprocess_video, batch_size=16)

# Create data generators
train_gen = data_generator(train_paths, train_labels, preprocess_video, batch_size=16)
dev_gen = data_generator(dev_paths, dev_labels, preprocess_video, batch_size=16)
save_model_path = '/content/drive/My Drive/Final_Dataset/saved_model/my_model.h5'
# Train the model
history = model.fit(
    train_gen,
    steps_per_epoch=5,
    validation_data=dev_gen,
    validation_steps=5,
    epochs=10
)
model.save(save_model_path)
print(f"Model saved to: {save_model_path}")

Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 338s 76s/step - accuracy: 0.4875 - loss: 23.3319 - val_accuracy: 0.7250 - val_loss: 0.6809
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 254s 64s/step - accuracy: 0.6986 - loss: 0.6656 - val_accuracy: 0.7875 - val_loss: 0.5897
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 235s 59s/step - accuracy: 0.7462 - loss: 0.7091 - val_accuracy: 0.6875 - val_loss: 0.6422
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 230s 57s/step - accuracy: 0.6773 - loss: 0.6453 - val_accuracy: 0.7125 - val_loss: 0.6030
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 229s 57s/step - accuracy: 0.7413 - loss: 0.6021 - val_accuracy: 0.7250 - val_loss: 0.5901
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 226s 57s/step - accuracy: 0.8436 - loss: 0.4925 - val_accuracy: 0.8250 - val_loss: 0.4615
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 224s 56s/step - accuracy: 0.7521 - loss: 0.5485 - val_accuracy: 0.6875 - val_loss: 0.6359
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 226s 57s/step - accuracy: 0.7493 - loss: 0.6143 - val_accuracy: 0.6625 - val_loss

Model saved to: /content/drive/My Drive/Final_Dataset/saved_model/my_model.h5


In [ ]:
data_directory = '/content/drive/My Drive/Final_Dataset'


# Number of test steps
test_steps = len(test_paths) // 16
test_pred = model.predict(test_gen)
test_loss, test_accuracy = model.evaluate(test_data, test_pred )
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

      9/Unknown 290s 36s/step

# New Section